In [1]:
# %%capture
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


KeyboardInterrupt: 

# Agents

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate


# Load OpenAI's LLMs with LangChain
os.environ["OPENAI_API_KEY"] = ""
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [ ]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### ReAct Demo Code 2

In [ ]:
%%capture
# update or install the necessary libraries
!pip install --upgrade langchain
!pip install --upgrade python-dotenv
!pip install google-search-results


# import libraries
import openai
import os
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from dotenv import load_dotenv
load_dotenv()

# load API keys; you will need to obtain these if you haven't yet
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPER_API_KEY"] = ""

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
!pip install openai==0.28

from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
tools = load_tools(["google-serper", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

/tmp/ipython-input-2-54340317.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
/tmp/ipython-input-2-54340317.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_ag

In [ ]:
# run the agent
agent.run("Who are Dr. Raj Dandekar's co-founders? What is their current age raised to the 1 power?")

/tmp/ipython-input-3-2843818337.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Who are Dr. Raj Dandekar's co-founders? What is their current age raised to the 1 power?")




> Entering new AgentExecutor chain...
I can use a search engine to find information about Dr. Raj Dandekar's co-founders.
Action: google_serper
Action Input: "Dr. Raj Dandekar co-founders"
Observation: Dr. Raj Abhijit Dandekar. MIT PhD, IIT Madras BTech. Co-founder, Vizuara AI Labs ; Dr. Rajat Dandekar. Purdue PhD, IIT Madras BTech Co-founder, Vizuara AI Labs ... Founders. Dr. Raj Dandekar. IIT Madras BTech, MIT PhD. Dr. Rajat Dandekar. IIT Madras MTech, Purdue PhD. Dr. Sreedath Panat. IIT Madras BTech, MIT PhD ... My name is Dr. Raj Dandekar, one of the 3 co-founders of Vizuara AI Labs. I graduated with a PhD from Massachusetts Institute of Technology ... Industry Presentation: Raj Dandekar, Co Founder, Vizuara · Comments. I am currently the co-founder of two startups Vizuara and Videsh. At Vizuara, we are on a… · Experience: Vizuara · Education: Purdue University · Location: ... As the Co-Founder of Vizuara and Videsh, Raj is responsible for spearheading the strategic vision, produ

"The current age of Dr. Raj Dandekar's co-founders raised to the power of 1 is 2."